In [36]:
import tensorflow as tf
import numpy as np
import os
import glob

path_to_file = tf.keras.utils.get_file(
    'shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
)
text = open(path_to_file, 'r').read()
print(f"Length of text: {len(text)} characters")


Length of text: 1115394 characters


In [37]:
vocab = sorted(set(text))
vocab_size = len(vocab)
print(f"Unique characters: {vocab_size}")

char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

seq_length = 50 
examples_per_epoch = len(text)//seq_length

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

BATCH_SIZE = 32
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

Unique characters: 65


In [46]:
embedding_dim = 128
rnn_units = 256
EPOCHS = 5 

def build_model(vocab_size, embedding_dim, rnn_units, batch_size, model_type='gru', stateful=False):
    RNNLayer = {
        'rnn': tf.keras.layers.SimpleRNN,
        'lstm': tf.keras.layers.LSTM,
        'gru': tf.keras.layers.GRU
    }[model_type]
    
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        RNNLayer(units=rnn_units, return_sequences=True, stateful=stateful,
                 recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE, model_type='gru', stateful=False)

In [39]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [40]:
checkpoint_dir = './training_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_epoch_{epoch:02d}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
    save_freq='epoch'
)

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/5
683/683 ━━━━━━━━━━━━━━━━━━━━ 64s 88ms/step - loss: 2.1827
Epoch 2/5
683/683 ━━━━━━━━━━━━━━━━━━━━ 61s 88ms/step - loss: 1.7092
Epoch 3/5
683/683 ━━━━━━━━━━━━━━━━━━━━ 61s 88ms/step - loss: 1.5727
Epoch 4/5
683/683 ━━━━━━━━━━━━━━━━━━━━ 61s 89ms/step - loss: 1.5047
Epoch 5/5
683/683 ━━━━━━━━━━━━━━━━━━━━ 65s 93ms/step - loss: 1.4620


In [42]:
def build_stateful_model(vocab_size, embedding_dim, rnn_units, batch_size=1, model_type='gru'):
    RNNLayer = {
        'rnn': tf.keras.layers.SimpleRNN,
        'lstm': tf.keras.layers.LSTM,
        'gru': tf.keras.layers.GRU
    }[model_type]
    
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        RNNLayer(units=rnn_units, return_sequences=True, stateful=True,
                 recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

gen_model = build_stateful_model(vocab_size, embedding_dim, rnn_units, batch_size=1, model_type='gru')

In [47]:
gen_model.build(tf.TensorShape([1, None]))

weights_files = glob.glob(os.path.join(checkpoint_dir, "*.weights.h5"))
weights_files.sort()
latest_checkpoint = weights_files[-1]

print("Loading weights from:", latest_checkpoint)
gen_model.load_weights(latest_checkpoint)

Loading weights from: ./training_checkpoints\ckpt_epoch_05.weights.h5


In [44]:
def generate_text(model, start_string, char2idx, idx2char, generation_length=500, temperature=1.0):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []

    for layer in model.layers:
        if hasattr(layer, "reset_states"):
            layer.reset_states()
    
    for _ in range(generation_length):
        predictions = model(input_eval)
        predictions = predictions[:, -1, :]
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        text_generated.append(idx2char[predicted_id])
        input_eval = tf.expand_dims([predicted_id], 0)
    
    return start_string + ''.join(text_generated)

In [45]:
start_string = "To be, or not to be, "
generated_text = generate_text(gen_model, start_string, char2idx, idx2char,
                               generation_length=500, temperature=0.8)

print(generated_text)

To be, or not to be, let him in the prosperence of the and
Off my lord, if we are the stance for a mistress,
But that he made a strong is the sunses of him, which is them better
To be thee so farshed my his business, the our love,
That word screptestant masters, touchey my honours execial rage,
Which the held and once lauds of him the sond.

BIONDELO:
Masher as you so him open to thing.

First Servant:
For thy mad, ere had they and this confess both the honour.

DUKE OF YORK:
What is madam; and in part to be knowing
